In [1]:
from mast3r_slam.mast3r_utils import (
    load_mast3r,
    load_retriever,
    mast3r_inference_mono,
)
from mast3r_slam.mast3r_utils import mast3r_match_asymmetric

model = load_mast3r(device="cuda:0")
# model.share_memory()

/root/miniconda3/envs/mast3r-slam/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


... loading model from checkpoints/MASt3R_ViTLarge_BaseDecoder_512_catmlpdpt_metric.pth


/workspace/RoMA-SLAM/thirdparty/mast3r/mast3r/model.py:24: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(model_path, map_location='cpu')


instantiating : AsymmetricMASt3R(enc_depth=24, dec_depth=12, enc_embed_dim=1024, dec_embed_dim=768, enc_num_heads=16, dec_num_heads=12, pos_embed='RoPE100',img_size=(512, 512), head_type='catmlp+dpt', output_mode='pts3d+desc24', depth_mode=('exp', -inf, inf), conf_mode=('exp', 1, inf), patch_embed_cls='PatchEmbedDust3R', two_confs=True, desc_conf_mode=('exp', 0, inf), landscape_only=False)
<All keys matched successfully>


In [2]:
import torch
import numpy as np
from PIL import Image
import time
import matplotlib.pyplot as plt

imA = np.array(Image.open("datasets/tum/rgbd_dataset_freiburg2_xyz/rgb/1311867170.462290.png"))
imA = imA.transpose(2,0,1)[None,...]
imA = torch.from_numpy(imA / 255).float().cuda()

imB = np.array(Image.open("datasets/tum/rgbd_dataset_freiburg2_xyz/rgb/1311867220.809727.png"))
imB = imB.transpose(2,0,1)[None,...]
imB = torch.from_numpy(imB/255).float().cuda()


In [3]:
def decoder(model, feat1, feat2, pos1, pos2, shape1, shape2):
    dec1, dec2 = model._decoder(feat1, pos1, feat2, pos2)
    with torch.amp.autocast(enabled=False, device_type="cuda"):
        res1 = model._downstream_head(1, [tok.float() for tok in dec1], shape1)
        res2 = model._downstream_head(2, [tok.float() for tok in dec2], shape2)
    return res1, res2


start_time = time.time()

feat1, pos1, _ = model._encode_image(
    imA, imA.shape
)
feat2, pos2, _ = model._encode_image(
    imB, imB.shape
)

shape1, shape2 = imA.shape, imB.shape

elapsed_time = time.time() - start_time
print(f"Time spent: {elapsed_time:.4f} seconds")
res11, res21 = decoder(model, feat1, feat2, pos1, pos2, shape1, shape2)
res = [res11, res21]
X, C, D, Q = zip(
    *[(r["pts3d"][0], r["conf"][0], r["desc"][0], r["desc_conf"][0]) for r in res]
)
# 4xhxwxc
X, C, D, Q = torch.stack(X), torch.stack(C), torch.stack(D), torch.stack(Q)
X, C, D, Q = downsample(X, C, D, Q)

Time spent: 0.2448 seconds


OutOfMemoryError: CUDA out of memory. Tried to allocate 66.00 MiB. GPU 0 has a total capacity of 11.66 GiB of which 37.06 MiB is free. Process 4179807 has 11.61 GiB memory in use. Of the allocated memory 11.17 GiB is allocated by PyTorch, and 146.85 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)